In [2]:
# Load the CIFAR-10 dataset
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np

In [3]:
# Split the data between the test and training sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)

170500096/170498071 [==============================] - 83s 0us/step
x_train shape:  (50000, 32, 32, 3)
x_test shape:  (10000, 32, 32, 3)
